In [32]:
#look up and read papers on comparing SQL queries
##how do people say 2 queries are similar or not
#find out queries with same set of operators but different parameters (where clause)
#how many queries are sub queries of other queries

import matplotlib.pyplot as plt
import collections
import re
import numpy as np
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where, Token, Function
from sqlparse.tokens import Keyword, DML, Whitespace, Newline, Name, Comparison, Operator
from collections import OrderedDict


def is_subselect(parsed):
    '''
    @param parsed: a parsed SQL query
    @return: True if there is subquery or False if there is not
    '''
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

def clean_up(line):
    '''SQLparse does not properly parse sql query if some keywords are immediately followed by
        parenthesis. it adds a space between them
    @param line: string that is part of a sql query
    @reutrn line: cleaned up string
    '''
    line = line.upper()
    line = line.replace("SELECT("," SELECT (")
    line = line.replace("FROM(", " FROM (")
    line = line.replace("WHERE(", " WHERE (")
    line = line.replace("ORDER BY", " ORDER BY ")
    line = line.replace("GROUP BY", " GROUP BY ")
    line = line.replace("LINE(", "LINE (")
    return line


def extract_from_part(parsed):
    '''
    @param parsed: a parsed SQL query
    @return: string of tokens in the FORM clause
    '''
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if item.ttype == Newline or item.ttype == Whitespace:
                continue
            else:
                if isinstance(item,IdentifierList):
                    #res = []
                    #for each in item:
                    #    if isinstance(each,Identifier):
                    #        res.append(each._get_repr_name().lstrip().rstrip())
                    #return ",".join(res)
                    res = []
                    for each in item:
                        res.append(each.value)
                    return ",".join(res)
                elif isinstance(item, Identifier):
                    return item.get_real_name()
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True
            
def extract_where_part(parsed):
    '''
    @param parsed: a parsed SQL query
    @return: list of tokens in WHERE clause
    '''
    for item in parsed.tokens:
        if isinstance(item, Where):
            return item
    return None

def populate_dictionary(file_name, table_dict):
    '''
    populate dictionary with WHERE clause being its key and value being the list of queries with same WHERE clause
    @param file_name: file name containing queries
    @param table_dict: dictionary to store queries with the key being FROM clause
    '''
    queries = get_queries(file_name)
    
    for query in queries:
        parsed = sqlparse.parse(query)[0]
        from_part_key = extract_from_part(parsed)
        if table_dict.get(from_part_key) is None:
            table_dict[from_part_key] = set()
            table_dict[from_part_key].add(query)
        else:
            table_dict[from_part_key].add(query)

def get_queries(file_name):
    '''
    @param file_name: name of the file to be processed
    @return: list of string of queries
    '''

    f = open(file_name,"r")

    queries = []
    current_query =""
    
    #removing any extra white spaces from the queries and put them in a list "queries" 
    for line in f:
        line = line.lstrip().rstrip()
        # ignore comments
        if len(line) > 2 and line[0:2] == "--":
            continue
        if line != "" and line[0] != "_":
            line = " " + clean_up(line) + " "
            current_query += line
        if line!= "" and line[0] == "_":
            if current_query != "":
                queries.append(current_query)
                current_query =""
    
    f.close()
    return queries

def clean_tokenlist(some_list):
    '''
    @param some_list: list of tokens
    @return: list of tokens without any Newline and Whitespace
    '''
    return [x for x in some_list.tokens if x.ttype is not Whitespace and x.ttype is not Newline]

def create_dict_from_where(wh):
    op_param_map = {}
    for index, token in enumerate(wh):
        key  = value = None
        if token.ttype is Keyword and token.value.upper() == "BETWEEN":
            key = "" + wh[index-1].value.upper() + " " + "BETWEEN"
            value = "" + wh[index+1].value.upper() + "_" + wh[index+3].value   
        if token.ttype is Keyword and token.value.upper() == "LIKE":
            key = "" + wh[index - 1].value.upper() + " "  + "LIKE"
            value = wh[index + 1].value
        if token.ttype is Keyword and token.value.upper() == "IN":
            if wh[index-1].ttype is Keyword and wh[index-1].value.upper() == "NOT":
                key = "" + wh[index-2].value + " NOT IN"
                value = wh[index+1].value
            else:
                key = "" + wh[index-1].value + " IN"
                value = wh[index+1].value
        if token.ttype is Keyword and token.value.upper() == "IS":
            key = "" + wh[index-1].value + " " + "IS"
            value = "" + wh[index+1].value
        if token._get_repr_name() is "Comparison":
            comp_tokens = clean_tokenlist(token)
            key = "" + comp_tokens[0].value + " " + comp_tokens[1].value
            value = "" + comp_tokens[2].value
        if key is not None:
            op_param_map[key] = value;
        else:
            continue
    return op_param_map
    
def compute_where_similarity(where1, where2):
    '''
    true if two Where clauses have same operators but different parameters
    @param where1: list of tokens of a query
    @param where2: list of tokens of another query
    return: True if they have same operators but diff parameters or False o.w
    '''
    where1 = clean_tokenlist(where1)
    where2 = clean_tokenlist(where2)
    if len(where1) != len(where2):
        return False
    else:
        where1_dict = create_dict_from_where(where1)
        where2_dict = create_dict_from_where(where2)
        key_set_where1 = set(list(where1_dict.keys()))
        key_set_where2 = set(list(where2_dict.keys()))
        if key_set_where1 == key_set_where2:
            value_set_where1 = set(list(where1_dict.values()))
            value_set_where2 = set(list(where2_dict.values()))
            if value_set_where1 != value_set_where2:
                return True
            else:
                return False
        else:
            return False
        
    



table_dict = {}

populate_dictionary('queries.txt',table_dict)

tables_with_prop = 0
num_tables = 0
for table, queries  in table_dict.items():
    num_tables += 1
    if len(queries) > 1:
        similar_queries = set()
        queries = list(queries)
        for i in range(len(queries)):
            for j in range(i+1, len(queries)):          
                parsed1 = sqlparse.parse(queries[i])[0]
                parsed2 = sqlparse.parse(queries[j])[0]
                where1_part = extract_where_part(parsed1)
                where2_part = extract_where_part(parsed2)
                if where1_part is None or where2_part is None:
                    continue
                elif compute_where_similarity(where1_part, where2_part):
                    similar_queries.add(queries[i])
                    similar_queries.add(queries[j])
        #print(len(similar_queries))
        #print(queries)
        #if len(similar_queries) > 1:
        #    print("" + len(similar_queries) + "/" + len(queries))
        #    for query in similar_queries:
        #        print(query)
        #    print('---------',end='\n')
        if len(similar_queries) > 0:
            tables_with_prop += 1
            for query in similar_queries:
                print(query)
            print(str(len(similar_queries))+ "/" + str(len(queries)))

print("Number of tables with at least 2 queries that has same operators but different parameters/Total Number of tables")
print(str(tables_with_prop) +"/" + str(num_tables))






 SELECT T1.SPECIES,T2.SPECIES  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITUDE-T2.LONGITUDE,2)) < 0.01 
 SELECT T1.SPECIES,T2.SPECIES  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITUDE-T2.LONGITUDE,2)) < 0.001 
 SELECT COUNT(T1.SPECIES)  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITUDE-T2.LONGITUDE,2)) < 0.001 
 SELECT COUNT(T1.SPECIES)  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITUDE-T2.LONGITUDE,2)) < 0.0001 
 SELECT COUNT(T1.SPECIES)  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITUDE-T2.LONGITUDE,2)) < 0.000001 
 SELECT T1.SPECIES,T2.SPECIES  FROM [354].[OR3COL_POS] T1  , [354].[OR3COL_POS] T2  WHERE SQRT(POWER(T1.LATITUDE-T2.LATITUDE,2)+POWER(T1.LONGITU

 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTER_RV.6157841]   GROUP BY  FOLLOWER,FOLLOWEE  ) GROUPED  WHERE X < 10000000 
 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTER_RV.6157841]   GROUP BY  FOLLOWER,FOLLOWEE  ) GROUPED  WHERE X < 10000 
 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTER_RV.6157841]   GROUP BY  FOLLOWER,FOLLOWEE  ) GROUPED  WHERE X < 100000 
 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTER_RV.6157841]   GROUP BY  FOLLOWER,FOLLOWEE  ) GROUPED  WHERE X < 1000 
 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTER_RV.6157841]   GROUP BY  FOLLOWER,FOLLOWEE  ) GROUPED  WHERE X < 1000000 
 SELECT COUNT(*)  FROM (  SELECT FOLLOWER AS X,FOLLOWEE AS Y,MIN(FOLLOWER) AS IGNORED  FROM [354].[TWITTE

 SELECT  CHR AS SEQNAME,  'METHRATIO' AS SOURCE,  'CPG' AS FEATURE,  POS AS START,  POS + 1 AS [END],  CAST(RATIO AS FLOAT) AS SCORE,  STRAND,  '.' AS FRAME,  '.' AS ATTRIBUTE  FROM [1123].  [BIGILL_METHRATIO_V9_A.TXT]  WHERE  CONTEXT LIKE '__CG_'  AND CT_COUNT >= 5  AND RATIO <> 'NA'  AND CAST(RATIO AS FLOAT) >= 0.5 
 SELECT  CHR AS SEQNAME,  'METHRATIO' AS SOURCE,  'CPG' AS FEATURE,  POS AS START,  POS + 1 AS [END],  CAST(RATIO AS FLOAT) AS SCORE,  STRAND,  '.' AS FRAME,  '.' AS ATTRIBUTE  FROM [1123].  [BIGILL_METHRATIO_V9_A.TXT]  WHERE  CONTEXT LIKE '__CG_'  AND CT_COUNT >= 5  AND RATIO <> 'NA'  AND CAST(RATIO AS FLOAT) >= 0.500   ORDER BY  SCORE 
 SELECT  CHR AS SEQNAME,  'METHRATIO' AS SOURCE,  'CPG' AS FEATURE,  POS AS START,  POS + 1 AS [END],  CAST(RATIO AS FLOAT) AS SCORE,  STRAND,  '.' AS FRAME,  '.' AS ATTRIBUTE  FROM [1123].  [BIGILL_METHRATIO_V9_A.TXT]  WHERE  CONTEXT LIKE '__CG_'  AND CT_COUNT >= 5  AND RATIO <> 'NA'  AND CAST(RATIO AS FLOAT) >= 0.500 
3/36
 SELECT TOP 1

 SELECT * FROM [1017].[HEAD_GENES_WIF_1.TXT]WIF  LEFT JOIN  [1017].[HEAD_GENES_WAGE.TXT]WAGE  ON  WIF.HEADS=WAGE.HEAD  WHERE  HEAD='%FBGN' 
 SELECT * FROM [1017].[HEAD_GENES_WIF_1.TXT]WIF  LEFT JOIN  [1017].[HEAD_GENES_WAGE.TXT]WAGE  ON  WIF.HEADS=WAGE.HEAD  WHERE  HEAD = '*FB' 
 SELECT * FROM [1017].[HEAD_GENES_WIF_1.TXT]WIF  LEFT JOIN  [1017].[HEAD_GENES_WAGE.TXT]WAGE  ON  WIF.HEADS=WAGE.HEAD  WHERE  HEAD = '*FBGN' 
 SELECT * FROM [1017].[HEAD_GENES_WIF_1.TXT]WIF  LEFT JOIN  [1017].[HEAD_GENES_WAGE.TXT]WAGE  ON  WIF.HEADS=WAGE.HEAD  WHERE  HEAD='*FBGN' 
 SELECT * FROM [1017].[HEAD_GENES_WIF_1.TXT]WIF  LEFT JOIN  [1017].[HEAD_GENES_WAGE.TXT]WAGE  ON  WIF.HEADS=WAGE.HEAD  WHERE  HEAD='%FBGN%' 
5/6
 SELECT * FROM [1017].[MATCHED_DEGS_IFHEADS_2]  WHERE  HEADS='~FBGN' 
 SELECT * FROM [1017].[MATCHED_DEGS_IFHEADS_2]  WHERE  HEADS='FBGN' 
 SELECT * FROM [1017].[MATCHED_DEGS_IFHEADS_2]  WHERE  HEADS='%FBGN' 
 SELECT * FROM [1017].[MATCHED_DEGS_IFHEADS_2]  WHERE  HEADS='&FBGN' 
 SELECT * FROM

 SELECT * FROM [1117].[ORCAMASTER2010.CSV]  WHERE YEAR BETWEEN 1991 AND 1992   ORDER BY  YEAR, MONTH, DAY 
 SELECT DISTINCT SIGHTDATE, MONTH, DAY, [YEAR] FROM [1117].[ORCAMASTER2010.CSV]  WHERE [YEAR] BETWEEN 1990 AND 1992 AND POD <> 'TS?' AND MONTH BETWEEN 4 AND 10   ORDER BY  YEAR, MONTH, DAY 
 SELECT * FROM [1117].[ORCAMASTER2010.CSV]  WHERE LAT BETWEEN 48.2961 AND 48.2817 
 SELECT DISTINCT SIGHTDATE, MONTH, DAY, [YEAR] FROM [1117].[ORCAMASTER2010.CSV]  WHERE [YEAR] BETWEEN 1993 AND 1995 AND POD <> 'TS?' AND MONTH BETWEEN 4 AND 10   ORDER BY  YEAR, MONTH, DAY 
 SELECT DISTINCT SIGHTDATE, MONTH, DAY, [YEAR] FROM [1117].[ORCAMASTER2010.CSV]  WHERE [YEAR] BETWEEN 2006 AND 2010 AND POD <> 'TS?' AND MONTH BETWEEN 4 AND 10 
 SELECT * FROM [1117].[ORCAMASTER2010.CSV]  WHERE MONTH BETWEEN 6 AND 10 
 SELECT * FROM [1117].[ORCAMASTER2010.CSV]  WHERE 1=1  AND [LONG] BETWEEN 123.0701 AND 123.0869 
 SELECT * FROM [1117].[ORCAMASTER2010.CSV]  WHERE LAT BETWEEN 48.2727 AND 48.2870 
 SELECT * FROM 

...

18/18
 SELECT * FROM [446].[FUNCTION_MAP]  WHERE [FUNCTION] LIKE '%RAMP%' 
 SELECT * FROM [446].[FUNCTION_MAP]  WHERE [FUNCTION] LIKE '%NRAMP%' 
2/2
 SELECT [TIME], [OCEAN_TMP] FROM [1059].[SEAFLOW ALL QUERY] WHERE [TIME] <= '12/9/2014 12:15:03 AM' 
 SELECT [TIME], [OCEAN_TMP] FROM [1059].[SEAFLOW ALL QUERY] WHERE [TIME] <= '12/9/2014 12:15:03' 
 SELECT [TIME], [OCEAN_TMP] FROM [1059].[SEAFLOW ALL QUERY] WHERE [TIME] <= '12/9/2014 00:15:03' 
3/20
 SELECT  A.[TIME], SALINITY, OCEAN_TMP, PAR, [VELOCITY (KNOTS)] AS VELOCITY,  B.PROCHLORO AS PROCHLORO_CONC, B.SYNECHO AS SYNECHO_CONC, B.PICOEUK AS PICOEUK_CONC, B.BEADS AS BEADS_CONC,  C.PROCHLORO AS PROCHLORO_SIZE, C.SYNECHO AS SYNECHO_SIZE, C.PICOEUK AS PICOEUK_SIZE, C.BEADS AS BEADS_SIZE,  (ISNULL(B.PROCHLORO, 0) + ISNULL(B.SYNECHO, 0) + ISNULL(B.PICOEUK, 0) + ISNULL(B.BEADS, 0)) AS TOTAL_CONC  FROM  [1059].[SFL_VIEW] AS A,  [1059].[SEAFLOW: POPULATION-WISE CONCENTRATIONS] AS B,  [1059].[SEAFLOW: POPULATION-WISE SIZE (FSC_SMALL)] AS C,  [

 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('5.3.3.4',  '5.4.99.14',  '5.5.1.7') 
 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('2.5.1.74') 
 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('1.14.99.42') 
 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('1.1.1.47',  '2.7.1.40',  '2.7.1.45',  '3.1.1.17',  '4.1.2.14',  '4.2.1.39',  '5.1.3.3',  '5.4.2.1') 
 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('2.1.1.114') 
 SELECT * FROM [SSGCID_EC_MAP]  WHERE EC_NUMBER IN  ('1.2.99.3') 
24/72
 SELECT TEP.PATHWAY_NAME, TEG.ENZYMES_SELECTED, TEG.GENUS, TEP.ENZYMES_IN_PATHWAY, (100*TEG.ENZYMES_SELECTED)/TEP.ENZYMES_IN_PATHWAY AS PERCENTAGE_COVERED  FROM [TOTAL_ENZYMES_PER_PATHWAY] TEP  JOIN [TOTAL_ENZYMES_PER_GENUS] TEG ON TEG.PATHWAY_NAME = TEP.PATHWAY_NAME  WHERE TEP.ENZYMES_IN_PATHWAY > 2   ORDER BY  TEG.GENUS, PERCENTAGE_COVERED DESC, TEP.ENZYMES_IN_PATHWAY DESC 
 SELECT TEP.PATHWAY_NAME, TEG.ENZYMES_SELECTED, TEG.GENUS, TEP.ENZYMES_IN_PATHWAY, (100*TEG.ENZYMES_SE

 SELECT  *  FROM [188].[TABLE_EC_PDB_GENUS.CSV]  WHERE LEN(GENUS) > 50 
 SELECT  *  FROM [188].[TABLE_EC_PDB_GENUS.CSV]  WHERE LEN(GENUS) > 49 
2/12
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '330 M-%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '330 M%6% ' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '330%-%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '330 M -%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '330 M%6%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '% -%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '% - %' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '%-%' 
 SELECT * FROM [TABLE_ISOLATE_ID_MAPPING.CSV]  WHERE ISOLATE_ID LIKE '% -% ' 
9/32
 SELECT DISTINCT P.EC_NUMBER, P.PATHWAY_ID, P.PATHWAY_NAME  FROM  [PATHWAYS_BY_SSGCID_ENZYME.CSV] P  JOIN SSGCID_EC_MAP S ON (S.EC_NUM

 SELECT LEFT([PROTEIN CODE-1], 5), [CRYSTALS?], [DIFFRACTION?], [DATA SET?]  FROM [XSTAL_TRACKER.CSV]  WHERE [PROTEIN CODE-1] LIKE 'MYTH%' 
 SELECT CLEAVED, COUNT(*)  FROM [XSTAL_TRACKER.CSV] X  RIGHT OUTER JOIN [PROTEINS_CLEAVED_PDB] P ON (X.[PROTEIN CODE-1] = P.SSGCIDID)  WHERE X.[INITIAL TRIALS SET UP] < '05-01-2011'  AND TOP_PDB <> ''   GROUP BY  CLEAVED 
 SELECT X.[CRYSTALS?], CLEAVED, COUNT(*)  FROM [XSTAL_TRACKER.CSV] X  LEFT JOIN PROTEINS_CLEAVED_PDB P ON (X.[PROTEIN CODE-1] = P.SSGCIDID)  WHERE [INITIAL TRIALS SET UP] < '08-01-2011'   GROUP BY  X.[CRYSTALS?], CLEAVED   ORDER BY  CLEAVED 
 SELECT [CRYSTALS?], COUNT ([PROTEIN CODE-1])  FROM [XSTAL_TRACKER.CSV]  WHERE [INITIAL TRIALS SET UP] < '08-01-2011'   GROUP BY  [CRYSTALS?] 
 SELECT LEFT([PROTEIN CODE-1], 5) AS ORGANISM, SUBSTRING([PROTEIN CODE-1], 7, 5) AS FAMILY,  [CRYSTALS?], [DIFFRACTION?], [DATA SET?]  FROM [XSTAL_TRACKER.CSV]  WHERE [PROTEIN CODE-1] LIKE 'MYTH%' 
 SELECT COUNT([PROTEIN CODE-1]), [DATA SET?]  FROM [XST

 SELECT * FROM [1317].[COFFEEEEEEEEEE]  WHERE COUNTRY IN('ADEN', 'AUSTRIA') 
 SELECT * FROM [1317].[COFFEEEEEEEEEE]  WHERE COUNTRY IN ('ADEN', 'AUSTRIA', 'CUBA') 
2/15
 SELECT * FROM [1314HOWE].[UW HIGHEST PAID EMPLOYEE BY DEPARTMENT]  WHERE MIDDLE ='A' 
 SELECT * FROM [1314HOWE].[UW HIGHEST PAID EMPLOYEE BY DEPARTMENT]  WHERE MIDDLE ='O' 
 SELECT * FROM [1314HOWE].[UW HIGHEST PAID EMPLOYEE BY DEPARTMENT]  WHERE MIDDLE ='T' 
3/6
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 <>'$550,004.23'  AND COLUMN1 <>'$500,000.23' 
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 <>'$550,004.23'  AND COLUMN1 <>'$550,000.23' 
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 NOT IN ('$550,004.23','$500,000.23','$523,000.23') 
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 NOT IN ('$550,004.23',  '$500,000.23',  '$523,000.23') 
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 NOT IN ('$550,004.23',  '$500,000.23',  '$523,000.23'  ) 
 SELECT * FROM [206].[SQLTEST5.TXT]  WHERE COLUMN1 NOT

In [ ]:


m = get_queries('test.txt')
for x in m:
    parsed = sqlparse.parse(x)[0]
    print(parsed.tokens)
    where_part = extract_where_part(parsed)
    print(where_part)





In [184]:
sql5 = '''
SELECT COUNT(t1.species)
FROM [354].[OR3col_pos] t1
   , [354].[OR3col_pos] t2
WHERE sqrt(power(t1.latitude-t2.latitude,2)+power(t1.longitude-t2.longitude,2)) < 0.000001
      AND t1.latitude is not null
      AND t2.latitude is not null

  '''


parsed3 = sqlparse.parse(sql5)[0]
wh = extract_where_part(parsed3)
wh = clean_tokenlist(wh)
print(wh)
y = create_dict_from_where(wh)
for k,v in y.items():
    print(k,v)

'''
wh = [x for x in wh.tokens if x.ttype is not Whitespace and x.ttype is not Newline]
print(wh)
op_param_map = {}
for index, token in enumerate(wh):
    key  = value = None
    if token.ttype is Keyword and token.value.upper() == "BETWEEN":
        key = "" + wh[index-1].value.upper() + " " + "BETWEEN"
        value = "" + wh[index+1].value.upper() + "_" + wh[index+3].value   
    if token.ttype is Keyword and token.value.upper() == "LIKE":
        key = "" + wh[index - 1].value.upper() + " "  + "LIKE"
        value = wh[index + 1].value
    if token.ttype is Keyword and token.value.upper() == "IN":
        if wh[index-1].ttype is Keyword and wh[index-1].value.upper() == "NOT":
            key = "" + wh[index-2].value + " NOT IN"
            value = wh[index+1].value
        else:
            key = "" + wh[index-1].value + " IN"
            value = wh[index+1].value
    if token._get_repr_name() is "Comparison":
        token = clean_tokenlist(token)
        key = "" + token[0].value + " " + token[1].value
        value = "" + token[2].value
    
    if key is not None:
        op_param_map[key] = value;
    else:
        continue
'''   
        
        
#for k,v in op_param_map.items():
#    print(k,"\n",v)
        

[<Keyword 'WHERE' at 0x10C1E0948>, <Comparison 'sqrt(p...' at 0x10C07A930>, <Keyword 'AND' at 0x10C24F648>, <Identifier 't1.lat...' at 0x10C07AF48>, <Keyword 'is' at 0x10C24FCA8>, <Keyword 'not nu...' at 0x10C24FD68>, <Keyword 'AND' at 0x10C24F7C8>, <Identifier 't2.lat...' at 0x10C07A570>, <Keyword 'is' at 0x10C24F2E8>, <Keyword 'not nu...' at 0x10BF25168>]
sqrt(power(t1.latitude-t2.latitude,2)+power(t1.longitude-t2.longitude,2)) < 0.000001
t1.latitude IS not null
t2.latitude IS not null


'\nwh = [x for x in wh.tokens if x.ttype is not Whitespace and x.ttype is not Newline]\nprint(wh)\nop_param_map = {}\nfor index, token in enumerate(wh):\n    key  = value = None\n    if token.ttype is Keyword and token.value.upper() == "BETWEEN":\n        key = "" + wh[index-1].value.upper() + " " + "BETWEEN"\n        value = "" + wh[index+1].value.upper() + "_" + wh[index+3].value   \n    if token.ttype is Keyword and token.value.upper() == "LIKE":\n        key = "" + wh[index - 1].value.upper() + " "  + "LIKE"\n        value = wh[index + 1].value\n    if token.ttype is Keyword and token.value.upper() == "IN":\n        if wh[index-1].ttype is Keyword and wh[index-1].value.upper() == "NOT":\n            key = "" + wh[index-2].value + " NOT IN"\n            value = wh[index+1].value\n        else:\n            key = "" + wh[index-1].value + " IN"\n            value = wh[index+1].value\n    if token._get_repr_name() is "Comparison":\n        token = clean_tokenlist(token)\n        key = 

In [163]:
set1 = ("=", "my LIKE", "asdfas")
set2 = ("=", "my LIKE", "asdfas")
print(set1 != set2)
def myfunc(myset):
    for x in myset:
        print(x)
print(len(set1))
myfunc(set1)

False
3
=
my LIKE
asdfas
